# Project1 量化金融信用评估实验

In [32]:
import numpy as np
import pandas as pd
import datetime as dt
import time

## Read the dataset files.

In [33]:
train_df = pd.read_csv('/Users/Optimus-Prime/Documents/我的文件/课程相关/量化金融信用与风控分析/950503878_6_量化金融信用与反欺诈项目/LoanStats3d_securev1.csv', low_memory=False) # please note that this is a large file, it may take a little bit longer to read


In [4]:
#test_dataset = "antifraud_proj1_test.csv"
#test_df = pd.read_csv(dir_prefix + test_dataset, low_memory=False)
#print 'Shape of test_df:', test_df.shape
#print 'Which fields are missing in the test dataset?'
#print set(sample_df.columns) - set(test_df.columns)

## 针对Train_df数据进行清理

In [34]:
train_df['term'] = train_df['term'].replace([' 60 months',' 36 months'],[1,0])#'term' 修改为0-1变量
train_df = train_df.drop(['id','member_id','issue_d','emp_title'], axis = 1)#删除id，member_id
train_df['emp_length']=train_df['emp_length'].replace(['1 year', '10+ years', '2 years', '3 years', '4 years',
                                 '5 years', '6 years', '7 years', '8 years', '9 years', 
                                 '< 1 year', 'n/a'],[1,10,2,3,4,5,6,7,8,9,0,0])
train_df['home_ownership'] = train_df['home_ownership'].replace(['MORTGAGE', 'RENT', 'OWN','ANY'],[0,0.5,1,0.5])
train_df['verification_status'] = train_df['verification_status'].replace(['Source Verified', 
                                                                             'Not Verified', 'Verified'],
                                                                           [0.5,0,1])
#修改各个字段为数字

train_df = train_df[(train_df['loan_status']=='Current') | (train_df['loan_status']=='Charged Off') | (train_df['loan_status']=='Fully Paid')]
train_df['loan_status'] = train_df['loan_status'].replace(['Current','Fully Paid','Charged Off',],[0,0,1])
train_df.index = range(len(train_df))
#修改因变量为数字

del train_df['pymnt_plan']#数据值只有“n”，故删除
del train_df['desc']#描述
train_df['purpose'] = train_df['purpose'].replace(['car','credit_card','debt_consolidation','educational',
                                                     'home_improvement','house','major_purchase','medical',
                                                     'moving','other','renewable_energy','small_business',
                                                     'vacation','wedding'],[0.6,0.2,1.0,0.2,0.4,0.4,0.6,0.6,0.4,0.2,0.6,0.8,0.2,0.4])
del train_df['title']#和‘purpose’字段一致
del train_df['zip_code']#借贷程序的zip code 前三位

#根据贷款各项金额大小按比例排序

#地区,我按照各地的人均GDP与总GDP，按照7:3的比例进行合计评分，并归一化
train_df['addr_state'] = train_df['addr_state'].replace(['DC', 'TX', 'PA', 'GA', 'FL', 'NY', 'CA', 'TN', 'KS', 'MA', 
                                                           'RI','OH', 'OR', 'HI', 'SC', 'MD', 'AZ', 'WI', 'VA', 'CO', 
                                                           'IN', 'LA','NC', 'NJ', 'MO', 'NM', 'IL', 'MI', 'SD', 'WA', 
                                                           'NH', 'VT', 'AL','MN', 'CT', 'DE', 'NE', 'WV', 'MT', 'NV',
                                                           'OK', 'WY', 'AR', 'KY','MS', 'ME', 'UT', 'ND', 'AK'],
                                                          [0.249494545,0.065555092,0.039895621,0.085444756,0.593361556,
                                                              0.190277765,0.278189313,1,0.227317672,0.192702231,0.155311576,
                                                              0.131447429,0.280336943,0.132540357,0.106452086,0.069307246,
                                                              0.15253082,0.302445071,0.21218416,0.030257468,0.137946485,
                                                              0.208228197,0.110989561,0,0.048207504,0.16335986,0.231921874,
                                                              0.161796013,0.117910048,0.2777898,0.050846682,0.094623649,
                                                              0.494175602,0.191919255,0.085791932,0.160341405,0.220064923,
                                                              0.111679786,0.046895947,0.10685987,0.109896491,0.441591068,
                                                              0.097945954,0.207944043,0.06716049,0.242232775,0.141853116,
                                                              0.013158262,0.213121431])
#sample_df['addr_state']

In [35]:
lenth = len(train_df)

In [36]:
year,month = [0]*lenth,[0]*lenth
for i in range(lenth):
    date = train_df['earliest_cr_line'][i].split('/')
    year[i] = float(date[0])
    month[i] = float(date[1])

In [37]:
train_df_keys = list(train_df.keys())
ear_num = 0
for i in range(len(train_df_keys)):
    if train_df_keys[i] == 'earliest_cr_line':
        ear_num = i
        break


In [42]:
del train_df['earliest_cr_line']
train_df.insert(18,'earliest_cr_line',pd.DataFrame([(2018-year[i])*12+12-month[i] for i in range(lenth)]))

In [8]:
train_df[['loan_status','mths_since_last_delinq']].describe()

,loan_status,mths_since_last_delinq
count,405055.000000,208129.000000
mean,0.067894,34.076976
std,0.251565,21.992834
min,0.000000,0.000000
25%,0.000000,15.000000
50%,0.000000,31.000000
75%,0.000000,50.000000
max,1.000000,176.000000


In [39]:
#上一次逾期距今的月数，nan代表没有逾期,取0，那么月份越高说明危险度越低，取月份数的倒数
#train_df['mths_since_last_delinq'].unique()
train_df['mths_since_last_delinq']=train_df['mths_since_last_delinq'].fillna(0)
#sample_df['mths_since_last_delinq']

In [18]:
#上一次有记录的月份数
train_df['mths_since_last_record'] = train_df['mths_since_last_record'].fillna(0)
#sample_df['mths_since_last_record'] 

In [19]:
train_df['revol_util'] = train_df['revol_util'].str.strip('%').astype(float)/100
#网上将百分比转化的方法：p_float = df['p_str'].str.strip("%").astype(float)/100
#sample_df['revol_util'] 

In [20]:
train_df['mths_since_recent_bc_dlq'] = train_df['mths_since_recent_bc_dlq'].fillna(0)

In [21]:
train_df['mths_since_recent_revol_delinq'] = train_df['mths_since_recent_revol_delinq'].fillna(0)

In [22]:
del train_df['Unnamed: 0']

In [23]:
train_df_loan_status = train_df.loan_status
train_df = train_df.drop('loan_status',axis = 1)
train_df.insert(0,'loan_status',train_df_loan_status)

### 初步清洗完的数据集已导出到“clear_train_df.csv”

In [26]:
train_df.to_csv('clear_train_df.csv')

## 根据清理后的数据，进行归一化处理

In [27]:
cl_train_df = pd.read_csv('./clear_train_df.csv')

#del cl_train_df['Unnamed: 0']

#cl_train_df = cl_train_df.drop([i for i in cl_train_keys if cl_train_df[i].count()<100000 ],axis=1)

In [28]:
cl_train_keys = list(cl_train_df.keys())
num_non = [[i,len(cl_train_df)-cl_train_df[i].count()] for i in cl_train_keys]

#cl_train_df['num_tl_120dpd_2m'].describe()

In [31]:
[num_non[i] for i in range(len(num_non)) if num_non[i][1]>0]

[['revol_util', 156],
 ['open_acc_6m', 384509],
 ['open_il_6m', 384509],
 ['open_il_12m', 384509],
 ['open_il_24m', 384509],
 ['mths_since_rcnt_il', 385052],
 ['total_bal_il', 384509],
 ['il_util', 387175],
 ['open_rv_12m', 384509],
 ['open_rv_24m', 384509],
 ['max_bal_bc', 384509],
 ['all_util', 384509],
 ['inq_fi', 384509],
 ['total_cu_tl', 384509],
 ['inq_last_12m', 384509],
 ['bc_open_to_buy', 3765],
 ['bc_util', 4013],
 ['mo_sin_old_il_acct', 11837],
 ['mths_since_recent_bc', 3614],
 ['mths_since_recent_inq', 43442],
 ['num_rev_accts', 1],
 ['num_tl_120dpd_2m', 18365],
 ['percent_bc_gt_75', 4029]]

In [32]:
num_non = [i for i in num_non if i[1]>0]

name_non = [num_non[i][0] for i in range(len(num_non))]

for i in name_non:
    cl_train_df[i].fillna(0,inplace = True)

In [33]:
cl_train_df.to_csv('cl_train_df.csv')

In [104]:
cl_train_df[[i[0] for i in num_non if (i[1]>0)&(i[1]<100000)]].describe()

,revol_util,bc_open_to_buy,bc_util,mo_sin_old_il_acct,mths_since_recent_bc,mths_since_recent_inq,num_rev_accts,num_tl_120dpd_2m,percent_bc_gt_75
count,404899.000000,401290.000000,401042.000000,393218.000000,401441.000000,361613.000000,405054.000000,386690.000000,401026.000000
mean,0.537100,9759.249540,62.056067,127.656211,24.989134,6.782118,14.890773,0.000753,47.279265
std,0.238924,14911.049474,27.580544,52.005753,32.386066,5.924437,8.293047,0.030033,36.042640
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,0.359000,1372.000000,41.300000,101.000000,6.000000,2.000000,9.000000,0.000000,14.300000
50%,0.540000,4436.500000,65.600000,130.000000,14.000000,5.000000,13.000000,0.000000,50.000000
75%,0.720000,11750.000000,86.300000,153.000000,30.000000,10.000000,19.000000,0.000000,75.000000
max,1.930000,559912.000000,318.200000,724.000000,615.000000,25.000000,118.000000,6.000000,100.000000


## 利用箱形图计算各个特征的离群点数目

In [159]:
cl_train_keys = list(cl_train_df.keys())
#del cl_train_df['Unnamed: 0']
num_abnormal = [0]*len(cl_train_keys)
lenth = len(train_df)
for i in range(len(cl_train_keys)):
    Qu = cl_train_df[cl_train_keys[i]].describe()['75%']
    Ql = cl_train_df[cl_train_keys[i]].describe()['25%']
    IQR = Qu - Ql
    L = np.sort(cl_train_df[cl_train_keys[i]])
    j = 0
    while((L[j]<Ql-1.5*IQR) or L[lenth-1-j]>Qu+1.5*IQR):
        j += 1
        if L[j]<Ql-1.5*IQR:
            num_abnormal[i] += 1
        if L[lenth-1-j]>Qu+1.5*IQR:
            num_abnormal[i] += 1
print num_abnormal

[27500, 0, 0, 0, 0, 18401, 0, 0, 34800, 61, 83168, 11786, 11275, 11275, 16076, 1907, 71270, 10356, 71275, 24191, 35, 9151, 2292, 63658, 14600, 12563, 18307, 10090, 15214, 19990, 18361, 17747, 13709, 17760, 20156, 20523, 23987, 9523, 9507, 15344, 8934, 22234, 35859, 22, 3805, 1729, 26526, 10491, 33688, 26546, 5132, 38892, 92639, 8624, 39352, 98584, 9678, 13849, 19740, 8897, 25243, 8860, 11258, 12570, 10148, 273, 1539, 24201, 10821, 24760, 0, 48795, 15825, 14158, 23339, 24524, 20254]


In [189]:
pd.DataFrame(num_abnormal).to_csv('num_abnormal.csv')

## 检查之后，选取需要进行异常值处理的字段，形成abnormal_keys

In [194]:
abnormal_keys = ['dti','revol_util','bc_util','mo_sin_old_il_acct']

In [ ]:
cl_train_df[abnormal_keys].describe()

In [ ]:
for i in range(lenth):
    cl_train_df.loc[i,'dti'] = abs(cl_train_df['dti'][i])

In [259]:
np.random.seed(8)
for i in range(4):
    Qu = cl_train_df[abnormal_keys[i]].describe()['75%']
    Ql = cl_train_df[abnormal_keys[i]].describe()['25%']
    mean = cl_train_df[abnormal_keys[i]].describe()['mean']
    std = cl_train_df[abnormal_keys[i]].describe()['std']
    IQR = Qu - Ql
    cl_train_df = cl_train_df.sort_values(by = abnormal_keys[i])
    cl_train_df.index = range(lenth)
    j = 0
    while((cl_train_df[abnormal_keys[i]][j]<Ql-1.5*IQR) or cl_train_df[abnormal_keys[i]][lenth-1-j]>Qu+1.5*IQR):
        if cl_train_df[abnormal_keys[i]][j]<Ql-1.5*IQR:
            cl_train_df.loc[j,abnormal_keys[i]] = abs(float(np.random.normal(mean,std,1)))
        if cl_train_df[abnormal_keys[i]][lenth-1-j]>Qu+1.5*IQR:
            cl_train_df.loc[lenth-1-j,abnormal_keys[i]]  = abs(float(np.random.normal(mean,std,1)))
        j += 1

In [260]:
num_abnormal = [0]*4
for i in range(4):
    Qu = cl_train_df[abnormal_keys[i]].describe()['75%']
    Ql = cl_train_df[abnormal_keys[i]].describe()['25%']
    mean = cl_train_df[abnormal_keys[i]].describe()['mean']
    std = cl_train_df[abnormal_keys[i]].describe()['std']
    IQR = Qu - Ql
    cl_train_df = cl_train_df.sort_values(by = abnormal_keys[i])
    cl_train_df.index = range(lenth)
    j = 0
    while((cl_train_df[abnormal_keys[i]][j]<Ql-1.5*IQR) or cl_train_df[abnormal_keys[i]][lenth-1-j]>Qu+1.5*IQR):
        j += 1
        if cl_train_df[abnormal_keys[i]][j]<Ql-1.5*IQR:
            num_abnormal[i] += 1
        if cl_train_df[abnormal_keys[i]][lenth-1-j]>Qu+1.5*IQR:
            num_abnormal[i] += 1
num_abnormal

[0, 0, 0, 0]

In [243]:
cl_train_df[abnormal_keys[0]].describe()

count    405055.000000
mean         19.137358
std          23.899037
min           0.000000
25%          12.540000
50%          18.520000
75%          25.260000
max        9999.000000
Name: dti, dtype: float64

In [213]:
float(np.random.normal(0,1,1))

-1.724175884266502

In [158]:
len([cl_train_df['dti'][i] for i in range(lenth) if cl_train_df['addr_state'][i]>0.55])

34801

In [145]:
cl_train_df['loan_amnt'].describe()

count    405055.000000
mean      15213.303193
std        8571.239185
min        1000.000000
25%        8500.000000
50%       14000.000000
75%       20000.000000
max       35000.000000
Name: loan_amnt, dtype: float64

In [134]:
print cl_train_keys

['loan_status', 'loan_amnt', 'term', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl'

In [ ]:
for i in cl_train_keys:
    QU = np.percentile(cl_train_df[i],75)
    QL = np.percentile(cl_train_df[i],25)
    IQR = QU - QL
    num_abnormal.append([i,len([cl_train_df[i][j] for j in range(len(cl_train_df)) 
                                if (cl_train_df[i][j]<(QL-1.5*IQR))|(cl_train_df[i][j]>(QU+1.5*IQR))])])
num_abnormal

## Output the results
You do not need to modify the following lines. 

**Please do not modify the file name "antifraud_proj1_result.csv".**

In [12]:
d = {'Predict': np.zeros(test_df.shape[0])}
test_predict = pd.DataFrame(data=d)
test_predict.to_csv("antifraud_proj1_result.csv")